ls# Quiz 01

## Calculating parameters from intrinsic matrix using the Naïve Aproach

For doing so we will just take a picture of a rectangle, in this case the book's cover "100 years of Solitude" from Gabriel Garcia Marquez.

This method is called Naïve because we are calibrating the camera using only a few parameters,namely:
<div>
<img src="pictures/quiz01/naive_diagram.png" width="300"/>
</div>

Where dX and dY are the width and height of the book and dZ is the distance from the camera to the center of the book. The camera was aligned to the center of the book to the best of my capabilities, using a square, a ruler and the phone's internal accelerometer:
<br/><br/>
<div>
<img src="pictures/quiz01/alignment.jpg" width="300"/>
&nbsp;
<img src="pictures/quiz01/alignment2.jpg" width="300"/>
</div>

Once the picture was acquired the width and height of the book were mesured once again on the picture space in pixels and represented by dx and dy respectively:

<div>
<img src="pictures/quiz01/naive_calib_gimp.png" width="300"/>
</div>

<br/><br/>

$
(dX,dY,dZ) = (140mm,\ 210mm,\ 300mm), (dx, dy) = (1414px,\ 2123px) \\
(cols, rows) = (4032,\ 3024)\\

fx = \frac{dx}{dX}dZ = 3030px ,\ fy = \frac{dy}{dY}dZ = 3032px \\
$

So given the intersect matrix and doing the mentioned substitutions with the measured values we have:

<br/><br/>

$
M_{int} =\begin{pmatrix}
fx & 0  & \frac{cols}{2} \\
0  & fy & \frac{rows}{2}\\
0  & 0  & 1
\end{pmatrix} \Rightarrow

\begin{pmatrix}
3030  & 0      & 2016 \\
0     & 3032  & 1512\\
0     & 0      & 1
\end{pmatrix}
$

## Calibrating using MatLab Calibration Toolbox

<pre><code>
Calibration results after optimization (with uncertainties):

Focal Length:          fc = [ 2998.34967   2981.70766 ] +/- [ 181.26857   377.17289 ]
Principal point:       cc = [ 1388.25215   2005.80761 ] +/- [ 115.90141   280.28614 ]
Skew:             alpha_c = [ 0.00000 ] +/- [ 0.00000  ]   => angle of pixel axes = 90.00000 +/- 0.00000 degrees
Distortion:            kc = [ 0.18996   -1.12750   0.01047   -0.03334  0.00000 ] +/- [ 0.10398   1.00100   0.01878   0.01590  0.00000 ]
Pixel error:          err = [ 3.51625   1.73032 ]
</pre></code>

<img src="pictures/quiz01/matlab_error.png" width="300"/>
<img src="pictures/quiz01/matlab_load.png" width="300"/>
<img src="pictures/quiz01/matlab_marks.png" width="300"/>


## Python SVD Method

Here we will use python in order to calculate the parameters...

In [16]:
import numpy as np

wp = [[0, 0, 0],
      [3, 0, 0],
      [3, 0, 3],
      [0, 0, 3],
      [0, 3, 3],
      [0, 3, 0],
      [3, 3, 0]]

square_size = 30 #mm

wp = np.array(wp)
wp *= square_size


cp1 = [[1536.62, 1922.22, 0],
       [1236.71, 2071.60, 0],
       [1208.92, 1720.74, 0],
       [1530.83, 1588.73, 0],
       [1821.48, 1734.63, 0],
       [1800.64, 2091.29, 0],
       [1497.25, 2268.46, 0]]

cp2 = [[1562.71, 1688.58, 0],
       [1258.64, 1814.44, 0],
       [1224.11, 1426.82, 0],
       [1548.23, 1322.12, 0],
       [1843.40, 1426.82, 0],
       [1828.92, 1820.01, 0],
       [1517.05, 1973.72, 0]]

cp3 = [[1591.85, 2545.35, 0],
       [1421.69, 2631.44, 0],
       [1428.74, 2459.27, 0],
       [1603.43, 2379.22, 0],
       [1747.41, 2497.53, 0],
       [1728.28, 2671.72, 0],
       [1555.61, 2769.38, 0]]

cp1, cp2, cp3 = np.array(cp1), np.array(cp2), np.array(cp3)


In [14]:
no_points = len(wp)

calib_pics = [cp1, cp2, cp3]
A = []
for cp in calib_pics:
    for i in range(no_points):
        X, Y, Z = wp[i][0], wp[i][1], wp[i][2]
        u = cp[i][0]
        v = cp[i][1]

        A.append([X, Y, Z, 1, 0, 0, 0, 0, -u*Z, -u*Y, -u*Z, -u])
        A.append([0, 0, 0, 0, X, Y, Z, 1, -v*Z, -v*Y, -v*Z, -v])

np.matrix(A)

matrix([[ 0.000000e+00,  0.000000e+00,  0.000000e+00,  1.000000e+00,
          0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00,
         -0.000000e+00, -0.000000e+00, -0.000000e+00, -1.536620e+03],
        [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00,
          0.000000e+00,  0.000000e+00,  0.000000e+00,  1.000000e+00,
         -0.000000e+00, -0.000000e+00, -0.000000e+00, -1.922220e+03],
        [ 9.000000e+01,  0.000000e+00,  0.000000e+00,  1.000000e+00,
          0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00,
         -0.000000e+00, -0.000000e+00, -0.000000e+00, -1.236710e+03],
        [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00,
          9.000000e+01,  0.000000e+00,  0.000000e+00,  1.000000e+00,
         -0.000000e+00, -0.000000e+00, -0.000000e+00, -2.071600e+03],
        [ 9.000000e+01,  0.000000e+00,  9.000000e+01,  1.000000e+00,
          0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00,
         -1.088028e+05, -0.000